# Hyperparameter tuning with XGBoost, Ray Tune, Hyperopt and Optuna

## Introduction


In this post we are going to demonstrate how we can speed up hyperparameter tuning with:

1) Bayesian optimization tuning algos like HyperOpt and Optuna, running on…

2) the [Ray](https://ray.io/) distributed ML framework, with a [unified API to many hyperparameter search algos](https://medium.com/riselab/cutting-edge-hyperparameter-tuning-with-ray-tune-be6c0447afdf) with early stopping and…

3) a distributed cluster of cloud instances for even more speedup.

### Outline:
- Overview of hyperparameter tuning
- Baseline linear regression with no hyperparameters
- ElasticNet with L1 and L2 regularization using ElasticNetCV hyperparameter optimization
- ElasticNet with GridSearchCV hyperparameter optimization
- XGBoost: sequential grid search over hyperparameter subsets with early stopping 
- XGBoost: with HyperOpt and Optuna search algorithms
- LightGBM: with HyperOpt and Optuna search algorithms
- XGBoost: HyperOpt on a Ray cluster
- LightGBM: HyperOpt on a Ray cluster
- Concluding remarks
18735

But first, here are results on the Ames housing data set, predicting Iowa home prices:

| ML Algo           | Hyperparameter search algo   | CV Error (RMSE in $)  | Time     |
|-------------------|------------------------------|-----------------------|----------|
| XGB               | Sequential Grid Search       | 18783                |   36:09  |
| XGB               | HyperOpt (128 samples)       | 18770                | 13:36 |
| XGB               | Optuna (256 samples)         | 18722                | 43:21 |
| LightGBM          | HyperOpt (256 samples)       | 18612              |   45:08  |
| LightGBM          | Optuna                       |  18534               |  | 34:54
| XGB               | Optuna - 16-instance cluster | 18770                |   14:23  |
| LightGBM          | Optuna - 16-instance cluster | 18612                |    4:22  |
| Baseline:         |                              |
| Linear Regression | --                           | 18192                |   0:01s  |
| ElasticNet        | ElasticNetCV (Grid Search)   | 18122                |   0:02s  |          
| ElasticNet        | GridSearchCV                 | 18061                |   0:05s  |          

We see both speedup and RMSE improvement when using HyperOpt and Optuna, and the cluster. But our feature engineering was quite good and our simple linear model still outperforms boosting. (Not shown, SVR and KR are high-performing and an ensemble improves over all individual algos)


## Hyperparameter Tuning Overview

Here are [the principal approaches to hyperparameter tuning](https://en.wikipedia.org/wiki/Hyperparameter_optimization)

- Grid search: given a finite set of discrete values for each hyperparameter, exhaustively cross-validate all combinations

- Random search: given a discrete or continuous distribution for each hyperparameter, randomly sample from the joint distribution. Generally [more efficient than exhaustive grid search.](https://dl.acm.org/doi/10.5555/2188385.2188395 ) 

- Bayesian optimization: update the search space as you go based on outcomes of prior searches.

- Gradient-based optimization: attempt to estimate the gradient of the CV metric with respect to the hyperparameter and ascend/descend the gradient.

- Evolutionary optimization: sample the search space, discard combinations with poor metrics, and genetically evolve new combinations to try based on the successful combinations.

- Population-based: A method of performing hyperparameter optimization at the same time as training.

In this post we focus on Bayesian optimization with HyperOpt and Optuna. What is Bayesian optimization? When we perform a grid search, the search space can be considered a prior belief that the best hyperparameter vector is in the search space, and the combinations have equal probability of being the best combination. So we try them all and pick the best one.

Perhaps we might do two passes of grid search. After an initial search on a broad, coarsely spaced grid, we might do a deeper dive in a smaller area around the best metric from the first pass, with a more finely-spaced grid. In Bayesian terminology we updated our prior belief.

Bayesian optimization first samples randomly, e.g. 30 combinations, and computes the cross-validation metric for each combination. Then the algorithm updates the distribution it samples from, so it is more likely to sample combinations near the good metrics, and less likely to sample combinations near the poor metrics. As it continues to sample, it continues to update the search distribution based on the metrics it finds.

Early stopping may also highly beneficial: often we can discard a combination without fully training it. In this post we use [ASHA](https://arxiv.org/abs/1810.05934). 

We use 4 regression algorithms:
- LinearRegression: baseline with no hyperparameters
- ElasticNet: Linear regression with L1 and L2 regularization (2 hyperparameters).
- XGBoost
- LightGBM

We use 5 approaches :
- *Native CV*: In sklearn if an algo has hyperparameters it will often have an xxxCV version which performs automated hyperparameter tuning over a search space with specified kfolds.
- *GridSearchCV*: Abstracts CV for any sklearn algo, running multithreaded trials over specified folds. 
- *Manual sequential grid search*: What we typically do with XGBoost, which doesn't play well with GridSearchCV and has too many hyperparameters to tune in one pass.
- *Ray on local machine*: HyperOpt and Optuna with early stopping.
- *Ray on cluster*: Additionally scale out to run a single hyperparameter optimization task over many instances.

We use data from the Ames Housing Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques . The original data has 79 raw features. The data we will use has 100 features with a fair amount of feature engineering from [my own attempt at modeling](https://github.com/druce/iowa), which was in the top 5% or so when I submitted it to Kaggle.

### Further reading: 
 - [Hyper-Parameter Optimization: A Review of Algorithms and Applications](https://arxiv.org/abs/2003.05689) Tong Yu, Hong Zhu (2020)
 - [Hyperparameter Search in Machine Learning](https://arxiv.org/abs/1502.02127v2), Marc Claesen, Bart De Moor (2015)
 - [Hyperparameter Optimization](https://link.springer.com/chapter/10.1007/978-3-030-05318-5_1), Matthias Feurer, Frank Hutter (2019) 

In [1]:
from itertools import product
from datetime import datetime, timedelta
import os
import random
import string

import numpy as np
import pandas as pd

import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA

#!conda install -y -c conda-forge  xgboost 
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

import lightgbm
from lightgbm import LGBMRegressor

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.logger import DEFAULT_LOGGERS
# from ray.tune.integration.wandb import WandbLogger

# wandb is great for real-time tracking but not necessary
# import wandb
# from ray.tune.integration.wandb import wandb_mixin
# from wandb.lightgbm import wandb_callback
# os.environ['WANDB_NOTEBOOK_NAME']='hyperparameter_optimization'
# os.environ["WANDB_RUN_GROUP"] = "experiment-" + wandb.util.generate_id()
# wandb.init(group="experiment_1", job_type="eval")

print(datetime.now())

print ("%-20s %s"% ("numpy", np.__version__))
print ("%-20s %s"% ("pandas", pd.__version__))
print ("%-20s %s"% ("sklearn", sklearn.__version__))
print ("%-20s %s"% ("xgboost", xgboost.__version__))
print ("%-20s %s"% ("lightgbm", lightgbm.__version__))
print ("%-20s %s"% ("ray", ray.__version__))

pd.set_option('max_colwidth', 1000)


2020-10-18 10:51:17.716648
numpy                1.19.1
pandas               1.1.3
sklearn              0.23.2
xgboost              1.2.0
lightgbm             2.3.0
ray                  1.0.0


In [2]:
# set seed for reproducibility
RANDOMSTATE = 42
np.random.seed(RANDOMSTATE)


In [3]:
# import train data
df = pd.read_pickle('df_train.pickle')

response = 'SalePrice'
predictors = ['YearBuilt',
              'BsmtFullBath',
              'FullBath',
              'KitchenAbvGr',
              'GarageYrBlt',
              'LotFrontage',
              'MasVnrArea',
              '1stFlrSF',
              'GrLivArea',
              'GarageArea',
              'WoodDeckSF',
              'PorchSF',
              'AvgBltRemod',
              'FireBathRatio',
              'TotalSF x OverallQual x OverallCond',
              'AvgBltRemod x Functional x TotalFinSF',
              'Functional x OverallQual',
              'KitchenAbvGr x KitchenQual',
              'GarageCars x GarageYrBlt',
              'GarageQual x GarageCond x GarageCars',
              'HeatingQC x Heating',
              'monthnum',
              'log_YearBuilt',
              'log_LotArea',
              'log_TotalFinSF',
              'log_GarageRatio',
              'log_TotalSF x OverallQual x OverallCond',
              'log_TotalSF x OverallCond',
              'log_AvgBltRemod x TotalFinSF',
              'sq_2ndFlrSF',
              'sq_BsmtFinSF',
              'sq_BsmtFinSF x BsmtQual',
              'sq_BsmtFinSF x BsmtBath',
              'BldgType_4',
              'BsmtExposure_1',
              'BsmtExposure_4',
              'BsmtFinType1_1',
              'BsmtFinType1_2',
              'BsmtFinType1_4',
              'BsmtFinType1_5',
              'BsmtFinType1_6',
              'CentralAir_0',
              'CentralAir_1',
              'Condition1_1',
              'Condition1_3',
              'ExterCond_2',
              'ExterQual_2',
              'Exterior1st_4',
              'Exterior1st_5',
              'Exterior1st_10',
              'Fence_0',
              'Fence_2',
              'Foundation_1',
              'Foundation_5',
              'GarageCars_1',
              'GarageFinish_2',
              'GarageFinish_3',
              'GarageType_2',
              'HouseStyle_2',
              'KitchenQual_4',
              'LotConfig_0',
              'LotConfig_4',
              'MSSubClass_30',
              'MSSubClass_70',
              'MSZoning_0',
              'MSZoning_1',
              'MSZoning_4',
              'MasVnrType_2',
              'MasVnrType_3',
              'MoSold_1',
              'MoSold_5',
              'MoSold_6',
              'MoSold_11',
              'Neighborhood_3',
              'Neighborhood_4',
              'Neighborhood_5',
              'Neighborhood_10',
              'Neighborhood_11',
              'Neighborhood_16',
              'Neighborhood_17',
              'Neighborhood_19',
              'Neighborhood_22',
              'Neighborhood_24',
              'OverallCond_7',
              'OverallQual_5',
              'OverallQual_6',
              'OverallQual_7',
              'OverallQual_9',
              'PavedDrive_0',
              'PavedDrive_2',
              'SaleCondition_1',
              'SaleCondition_2',
              'SaleCondition_5',
              'SaleType_4',
              'BedroomAbvGr_1',
              'BedroomAbvGr_4',
              'BedroomAbvGr_5',
              'HalfBath_1',
              'TotalBath_1.0',
              'TotalBath_2.5']

display(df[predictors].head())
display(df[[response]].head())


,YearBuilt,BsmtFullBath,FullBath,KitchenAbvGr,GarageYrBlt,LotFrontage,MasVnrArea,1stFlrSF,GrLivArea,GarageArea,...,SaleCondition_1,SaleCondition_2,SaleCondition_5,SaleType_4,BedroomAbvGr_1,BedroomAbvGr_4,BedroomAbvGr_5,HalfBath_1,TotalBath_1.0,TotalBath_2.5
Id,,,,,,,,,,,,,,,,,,,,,
1,7,1,2,1,7,65.0,196.0,856,1710,548.0,...,0,0,0,1,0,0,0,1,0,0
2,34,0,2,1,34,80.0,0.0,1262,1262,460.0,...,0,0,0,1,0,0,0,0,0,1
3,9,1,2,1,9,68.0,162.0,920,1786,608.0,...,0,0,0,1,0,0,0,1,0,0
4,95,1,1,1,12,60.0,0.0,961,1717,642.0,...,1,0,0,1,0,0,0,0,0,0
5,10,1,2,1,10,84.0,350.0,1145,2198,836.0,...,0,0,0,1,0,1,0,1,0,0


,SalePrice
Id,
1,12.247699
2,12.109016
3,12.317171
4,11.849405
5,12.429220


In [4]:
# we are training on a response which is the log of 1 + the sale price
# transform prediction back to original basis with expm1 and evaluate vs. original

MEAN_RESPONSE=df[response].mean()
def cv_to_raw(cv_val, mean_response=MEAN_RESPONSE):
    """convert log1p rmse to underlying SalePrice error"""
    # MEAN_RESPONSE assumes folds have same mean response, which is true in expectation but not in each fold
    # we can also pass the actual response for each fold
    # but we're usually looking to consistently convert the log value to a more meaningful unit
    return np.expm1(mean_response+cv_val) - np.expm1(mean_response)

In [5]:
# always use same k-folds for reproducibility
kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOMSTATE)


## Baseline linear regression
- Raw CV RMSE 18191.9791

In [6]:
%%time
# Tune lr search space for alphas and l1_ratio
print("LinearRegression")

print(len(predictors), "predictors")

lr = LinearRegression()

# evaluate using kfolds
scores = -cross_val_score(lr, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


LinearRegression
100 predictors

Log1p CV RMSE 0.1037 (STD 0.0099)
Raw CV RMSE 18192 (STD 1839)
CPU times: user 76 ms, sys: 76 ms, total: 152 ms
Wall time: 1.04 s


## Native Sklearn xxxCV
- LogisticRegressionCV, LassoCV, RidgeCV, ElasticNetCV, etc.
- Test many hyperparameters in parallel with multithreading
- Note improvement vs. LinearRegression due to controlling overfitting
- RMSE $18103
- Time 5s


In [7]:
%%time
# Tune elasticnet search space for alphas and L1_ratio
# predictor selection used to create the training set used lasso
# so l1 parameter is close to 0
# could use ridge (eg elasticnet with 0 L1 regularization)
# but then only 1 param, more general and useful to do this with elasticnet
print("ElasticnetCV")

# make pipeline
# with regularization must scale predictors
elasticnetcv = make_pipeline(RobustScaler(),
                             ElasticNetCV(max_iter=100000, 
                                          l1_ratio=[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                          alphas=np.logspace(-4, -2, 9),
                                          cv=kfolds,
                                          n_jobs=-1,
                                          verbose=1,
                                         ))

#train and get hyperparams
elasticnetcv.fit(df[predictors], df[response])
l1_ratio = elasticnetcv._final_estimator.l1_ratio_
alpha = elasticnetcv._final_estimator.alpha_
print('l1_ratio', l1_ratio)
print('alpha', alpha)

# evaluate using kfolds on full dataset
# I don't see API to get CV error from elasticnetcv, so we use cross_val_score
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=10000)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.04f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


ElasticnetCV


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
.............................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.3s
...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

l1_ratio 0.01
alpha 0.0031622776601683794

Log1p CV RMSE 0.1030 (STD 0.0109)
Raw CV RMSE 18061 (STD 2008)
CPU times: user 5.93 s, sys: 3.67 s, total: 9.6 s
Wall time: 1.61 s


## GridSearchCV
- Useful for algos with no native multithreaded xxxCV
- Test many hyperparameter combinations in parallel with multithreading
- Similar result vs ElasticNetCV, not exact, need more research as to why


In [8]:
%%time
gs = make_pipeline(RobustScaler(),
                   GridSearchCV(ElasticNet(max_iter=100000),
                                param_grid={'l1_ratio': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
                                            'alpha': np.logspace(-4, -2, 9),
                                           },
                                scoring='neg_root_mean_squared_error',
                                refit=True,
                                cv=kfolds,
                                n_jobs=-1,
                                verbose=1
                               ))

# do cv using kfolds on full dataset
print("\nCV on full dataset")
gs.fit(df[predictors], df[response])
print('best params', gs._final_estimator.best_params_)
print('best score', -gs._final_estimator.best_score_)
l1_ratio = gs._final_estimator.best_params_['l1_ratio']
alpha = gs._final_estimator.best_params_['alpha']
print("Log1p CV RMSE %.06f" % (np.sqrt(-gs._final_estimator.best_score_)))
print("Raw CV RMSE %.0f" % (cv_to_raw(np.sqrt(-gs._final_estimator.best_score_))))

# we shouldn't need to do this
elasticnet = ElasticNet(alpha=alpha,
                        l1_ratio=l1_ratio,
                        max_iter=100000)
print(elasticnet)

scores = -cross_val_score(elasticnet, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))

# difference in average CV scores reported by GridSearchCV and cross_val_score
# with same alpha, l1_ratio, kfolds
# one reason could be that we used simple average, GridSearchCV is weighted by # of samples per fold?
nsamples = [len(z[1]) for z in kfolds.split(df)]
print("weighted average %.06f" % np.average(scores, weights=nsamples))
# still tiny difference, not sure why, also ElasticSearchCV shows fewer fits, takes less time



CV on full dataset
Fitting 10 folds for each of 117 candidates, totalling 1170 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 1170 out of 1170 | elapsed:    4.7s finished


best params {'alpha': 0.0031622776601683794, 'l1_ratio': 0.01}
best score 0.10247177583755482
Log1p CV RMSE 0.320112
Raw CV RMSE 62767
ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=100000)

Log1p CV RMSE 0.103003 (STD 0.0109)
Raw CV RMSE 18061 (STD 2008)
weighted average 0.103023
CPU times: user 2.08 s, sys: 612 ms, total: 2.7 s
Wall time: 5.14 s


In [9]:
# roll-our-own CV 
# matches cross_val_score
alpha = 0.0031622776601683794
l1_ratio = 0.01
regressor = ElasticNet(alpha=alpha,
                       l1_ratio=l1_ratio,
                       max_iter=10000)
print(regressor)
cverrors = []
for train_fold, cv_fold in kfolds.split(df): 
    fold_X_train=df[predictors].values[train_fold]
    fold_y_train=df[response].values[train_fold]
    fold_X_test=df[predictors].values[cv_fold]
    fold_y_test=df[response].values[cv_fold]
    regressor.fit(fold_X_train, fold_y_train)
    y_pred_test=regressor.predict(fold_X_test)
    cverrors.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
    
print("%.06f" % np.average(cverrors))
    

ElasticNet(alpha=0.0031622776601683794, l1_ratio=0.01, max_iter=10000)
0.103003


In [10]:
# try PCA
pipe = Pipeline(steps=[ ('scaler', RobustScaler()), ('pca', PCA()), ('elasticnet', ElasticNet(max_iter=10000))])

param_grid = {
    'pca__n_components': [5, 10, 15, 25, 50, 100],
    'elasticnet__l1_ratio': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99],
    'elasticnet__alpha': np.logspace(-4, -2, 9),
}

gs = GridSearchCV(pipe, param_grid,
                  scoring='neg_root_mean_squared_error',
                  refit=True,
                  cv=kfolds,
                  n_jobs=-1,
                  verbose=1)

gs.fit(df[predictors], df[response])
print('best params', gs.best_params_)
print('best score', -gs.best_score_)

pipe = Pipeline(steps=[('scaler', RobustScaler()),
                       ('pca', PCA(n_components=gs.best_params_['pca__n_components'])),  
                       ('elasticnet', ElasticNet(alpha=gs.best_params_['elasticnet__alpha'], 
                                                 l1_ratio=gs.best_params_['elasticnet__l1_ratio'], 
                                                 max_iter=10000))])
scores = -cross_val_score(pipe, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)
raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


Fitting 10 folds for each of 702 candidates, totalling 7020 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 584 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1584 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 2984 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 4784 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 6975 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 7020 out of 7020 | elapsed:   33.9s finished


best params {'elasticnet__alpha': 0.0017782794100389228, 'elasticnet__l1_ratio': 0.1, 'pca__n_components': 100}
best score 0.10225515641215403

Log1p CV RMSE 0.102255 (STD 0.0120)
Raw CV RMSE 17925 (STD 2213)


## XGBoost Sequential Grid Search 
- XGBoost has native multithreading, CV
- XGBoost has many tuning parameters so a complete grid search has an unreasonable number of combinations
- We tune reduced sets sequentially and use early stopping. 

### Tuning methodology
- Set an initial set of starting parameters
- Do 10-fold CV
- Use early stopping to halt training in each fold if no improvement after eg 100 rounds, pick hyperparameters to minimize average error over kfolds
- Tune sequentially on groups of hyperparameters that don't interact too much between groups to reduce combinations
- Tune max_depth and min_child_weight 
- Tune subsample and colsample_bytree
- Tune alpha, lambda and gamma (regularization)
- Tune learning rate: lower learning rate will need more rounds/n_estimators
- Retrain and evaluate on full dataset with best learning rate and best n_estimators (average stopping point over kfolds)

### Notes
- It doesn't seem possible to get XGBoost early stopping and also use GridSearchCV. GridSearchCV doesn't pass the kfolds in a way that XGboost understands for early stopping
- 2 alternative approaches 
    - use native xgboost .cv which understands early stopping but doesn't use sklearn API (uses DMatrix, not np array or dataframe)
    - use sklearn API and roll our own grid search instead of GridSearchCV (used below)
- XGboost terminology differs from sklearn
    - boost_rounds = n_estimators
    - eta = learning_rate
- parameter reference: https://xgboost.readthedocs.io/en/latest/parameter.html
- training reference: https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
- times are wall times on an amazon t2.2xlarge instance  
- to set up environment:
    - `conda create --name hyperparam python=3.8`
    - `conda activate hyperparam`
    - `conda install jupyter`
    - `pip install -r requirements.txt`
- round 1 Wall time: 6min 23s
- round 2 Wall time: 19min 22s
- round 3 Wall time: 5min 30s
- round 4 Wall time: 4min 54s
- total time 36:09
- RMSE 18783.117031

In [11]:
BOOST_ROUNDS=50000   # we use early stopping so make this arbitrarily high
EARLY_STOPPING_ROUNDS=100 # stop if no improvement after 100 rounds

def my_cv(df, predictors, response, kfolds, regressor, verbose=False):
    """Roll our own CV 
    train each kfold with early stopping
    return average metric, sd over kfolds, average best round"""
    metrics = []
    best_iterations = []

    for train_fold, cv_fold in kfolds.split(df): 
        fold_X_train=df[predictors].values[train_fold]
        fold_y_train=df[response].values[train_fold]
        fold_X_test=df[predictors].values[cv_fold]
        fold_y_test=df[response].values[cv_fold]
        regressor.fit(fold_X_train, fold_y_train,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                      eval_set=[(fold_X_test, fold_y_test)],
                      eval_metric='rmse',
                      verbose=verbose
                     )
        y_pred_test=regressor.predict(fold_X_test)
        metrics.append(np.sqrt(mean_squared_error(fold_y_test, y_pred_test)))
        best_iterations.append(regressor.best_iteration)
    return np.average(metrics), np.std(metrics), np.average(best_iterations)

def cv_over_param_dict(df, param_dict, predictors, response, kfolds, verbose=False):
    """given a list of dictionaries of xgb params
    run my_cv on params, store result in array
    return results
    """
    start_time = datetime.now()
    print("%-20s %s" % ("Start Time", start_time))

    results = []

    for i, d in enumerate(param_dict):
        xgb = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=BOOST_ROUNDS,
            random_state=RANDOMSTATE,    
            verbosity=1,
            n_jobs=-1,
            booster='gbtree',   
            **d
        )    

        metric_rmse, metric_std, best_iteration = my_cv(df, predictors, response, kfolds, xgb, verbose=False)    
        results.append([metric_rmse, metric_std, best_iteration, d])
    
        print("%s %3d result mean: %.6f std: %.6f, iter: %.2f" % (datetime.strftime(datetime.now(), "%T"), i, metric_rmse, metric_std, best_iteration))
        
    end_time = datetime.now()
    print("%-20s %s" % ("Start Time", start_time))
    print("%-20s %s" % ("End Time", end_time))
    print(str(timedelta(seconds=(end_time-start_time).seconds)))
    
    results_df = pd.DataFrame(results, columns=['rmse', 'std', 'best_iter', 'param_dict']).sort_values('rmse')
    display(results_df.head())
    
    best_params = results_df.iloc[0]['param_dict']
    return best_params, results_df

In [17]:
%%time
# set initial XGboost parameters
# run round 1 and get max_depth, min_child_weight based on best values 
# update parameters and run round 2 for subsample and colsample_bytree 
# update parameters and run round 3 for learning_rate and best n_iterations
# this is not an exhaustive list but a representative list of most important parameters to tune
# see https://xgboost.readthedocs.io/en/latest/parameter.html for all parameters
# https://sites.google.com/view/lauraepp/parameters

# early_stopping_rounds gives a warning but seems to be used
# seems like XGB doesn't like it when you use sklearn API beyond the basics
# possibly the native .cv might be the way to go

# initial hyperparams
current_params = {
    'max_depth': 5,
    'colsample_bytree': 0.5,
    'colsample_bylevel': 0.5,
    'subsample': 0.5,
    'learning_rate': 0.01,
}

##################################################
# round 1: tune depth , max_leaves
##################################################
max_depths = list(range(2,8))
# max_leavess = [1, 3, 10, 30, 100] #  doesn't matter
# grid_search_dicts = [dict(zip(['max_depth', 'max_leaves'], [a, b])) 
#                      for a,b in product(max_depths, max_leavess)]
grid_search_dicts = [{'max_depth': md} for md in max_depths]
# merge into full param dicts
full_search_dicts = [{**current_params, **d} for d in grid_search_dicts]

# cv and get best params
current_params, results_df = cv_over_param_dict(df, full_search_dicts, predictors, response, kfolds, verbose=False)

Start Time           2020-10-18 11:20:58.365236
11:21:26   0 result mean: 0.108542 std: 0.011448, iter: 2728.90
11:21:54   1 result mean: 0.108123 std: 0.011366, iter: 2225.30
11:22:29   2 result mean: 0.107936 std: 0.011714, iter: 2229.00
11:23:04   3 result mean: 0.108681 std: 0.011357, iter: 1884.00
11:23:41   4 result mean: 0.109373 std: 0.011591, iter: 1729.70
11:24:24   5 result mean: 0.109264 std: 0.011672, iter: 1740.10
Start Time           2020-10-18 11:20:58.365236
End Time             2020-10-18 11:24:24.034127
0:03:25


,rmse,std,best_iter,param_dict
2,0.107936,0.011714,2229.0,"{'max_depth': 4, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.5, 'learning_rate': 0.01}"
1,0.108123,0.011366,2225.3,"{'max_depth': 3, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.5, 'learning_rate': 0.01}"
0,0.108542,0.011448,2728.9,"{'max_depth': 2, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.5, 'learning_rate': 0.01}"
3,0.108681,0.011357,1884.0,"{'max_depth': 5, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.5, 'learning_rate': 0.01}"
5,0.109264,0.011672,1740.1,"{'max_depth': 7, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'subsample': 0.5, 'learning_rate': 0.01}"


CPU times: user 3min 24s, sys: 808 ms, total: 3min 25s
Wall time: 3min 25s


In [18]:
%%time
##################################################
# round 2: tune subsample and colsample_bytree
##################################################
# subsamples = np.linspace(0.01, 1.0, 10)
# colsample_bytrees = np.linspace(0.1, 1.0, 10)
# colsample_bylevel = np.linspace(0.1, 1.0, 10)
# narrower search
subsamples = np.linspace(0.25, 0.75, 11)
colsample_bytrees = np.linspace(0.1, 0.3, 5)
colsample_bylevel = np.linspace(0.1, 0.3, 5)
# subsamples = np.linspace(0.4, 0.9, 11)
# colsample_bytrees = np.linspace(0.05, 0.25, 5)

grid_search_dicts = [dict(zip(['subsample', 'colsample_bytree', 'colsample_bylevel'], [a, b, c])) 
                     for a,b,c in product(subsamples, colsample_bytrees, colsample_bylevel)]
# merge into full param dicts
full_search_dicts = [{**current_params, **d} for d in grid_search_dicts]
# cv and get best params
current_params, results_df = cv_over_param_dict(df, full_search_dicts, predictors, response, kfolds, verbose=False)


Start Time           2020-10-18 11:24:24.051740
11:24:42   0 result mean: 0.107415 std: 0.013652, iter: 3195.50
11:25:00   1 result mean: 0.107415 std: 0.013652, iter: 3195.50
11:25:15   2 result mean: 0.106196 std: 0.014139, iter: 2341.50
11:25:30   3 result mean: 0.106196 std: 0.014139, iter: 2341.50
11:25:43   4 result mean: 0.106210 std: 0.013387, iter: 1991.40
11:26:02   5 result mean: 0.107478 std: 0.014917, iter: 3299.40
11:26:17   6 result mean: 0.106020 std: 0.014010, iter: 2343.20
11:26:31   7 result mean: 0.106835 std: 0.013553, iter: 1999.70
11:26:44   8 result mean: 0.106835 std: 0.013553, iter: 1999.70
11:26:56   9 result mean: 0.107007 std: 0.014161, iter: 1728.80
11:27:10  10 result mean: 0.105374 std: 0.014054, iter: 2195.20
11:27:23  11 result mean: 0.105629 std: 0.013575, iter: 1997.30
11:27:36  12 result mean: 0.106201 std: 0.013386, iter: 1833.70
11:27:48  13 result mean: 0.107130 std: 0.013648, iter: 1669.20
11:28:01  14 result mean: 0.107105 std: 0.013030, iter: 

11:58:26 128 result mean: 0.105778 std: 0.013389, iter: 2405.50
11:58:42 129 result mean: 0.105735 std: 0.012839, iter: 2284.30
11:59:04 130 result mean: 0.106816 std: 0.014234, iter: 3550.30
11:59:22 131 result mean: 0.104776 std: 0.013353, iter: 2696.40
11:59:37 132 result mean: 0.106699 std: 0.013401, iter: 2004.50
11:59:51 133 result mean: 0.106699 std: 0.013401, iter: 2004.50
12:00:08 134 result mean: 0.106125 std: 0.013070, iter: 2192.50
12:00:26 135 result mean: 0.104839 std: 0.014065, iter: 2581.50
12:00:43 136 result mean: 0.105091 std: 0.012543, iter: 2430.80
12:01:00 137 result mean: 0.105268 std: 0.013035, iter: 2261.30
12:01:17 138 result mean: 0.105437 std: 0.012336, iter: 2021.20
12:01:34 139 result mean: 0.106073 std: 0.012405, iter: 2001.10
12:01:51 140 result mean: 0.106061 std: 0.013342, iter: 2562.40
12:02:07 141 result mean: 0.106388 std: 0.013287, iter: 2161.30
12:02:22 142 result mean: 0.106554 std: 0.012658, iter: 1939.70
12:02:40 143 result mean: 0.106346 std: 

12:37:21 257 result mean: 0.105659 std: 0.011889, iter: 2512.20
12:37:39 258 result mean: 0.105659 std: 0.011889, iter: 2512.20
12:37:56 259 result mean: 0.106025 std: 0.012595, iter: 2290.30
12:38:14 260 result mean: 0.105104 std: 0.013379, iter: 2727.60
12:38:31 261 result mean: 0.105822 std: 0.012719, iter: 2374.50
12:38:47 262 result mean: 0.105582 std: 0.012088, iter: 2191.00
12:39:02 263 result mean: 0.106008 std: 0.012499, iter: 1957.80
12:39:23 264 result mean: 0.105867 std: 0.012316, iter: 2478.10
12:39:41 265 result mean: 0.106334 std: 0.013229, iter: 2798.50
12:39:59 266 result mean: 0.106116 std: 0.012265, iter: 2482.30
12:40:16 267 result mean: 0.107133 std: 0.012923, iter: 2221.70
12:40:33 268 result mean: 0.106939 std: 0.012814, iter: 1981.90
12:40:52 269 result mean: 0.106546 std: 0.012301, iter: 2260.30
12:41:08 270 result mean: 0.106271 std: 0.013313, iter: 2243.10
12:41:26 271 result mean: 0.106144 std: 0.012457, iter: 2356.80
12:41:45 272 result mean: 0.106680 std: 

,rmse,std,best_iter,param_dict
36,0.104301,0.012940,2275.1,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.01}"
110,0.104553,0.013535,2666.7,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.1, 'subsample': 0.45, 'learning_rate': 0.01}"
86,0.104706,0.012578,2203.3,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.4, 'learning_rate': 0.01}"
61,0.104742,0.012613,2223.6,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.35, 'learning_rate': 0.01}"
131,0.104776,0.013353,2696.4,"{'max_depth': 4, 'colsample_bytree': 0.15, 'colsample_bylevel': 0.15, 'subsample': 0.5, 'learning_rate': 0.01}"


CPU times: user 1h 17min 31s, sys: 29.9 s, total: 1h 18min 1s
Wall time: 1h 18min 3s


In [19]:
results_df

,rmse,std,best_iter,param_dict
36,0.104301,0.012940,2275.1,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.01}"
110,0.104553,0.013535,2666.7,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.1, 'subsample': 0.45, 'learning_rate': 0.01}"
86,0.104706,0.012578,2203.3,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.4, 'learning_rate': 0.01}"
61,0.104742,0.012613,2223.6,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.35, 'learning_rate': 0.01}"
131,0.104776,0.013353,2696.4,"{'max_depth': 4, 'colsample_bytree': 0.15, 'colsample_bylevel': 0.15, 'subsample': 0.5, 'learning_rate': 0.01}"
...,...,...,...,...
48,0.107545,0.012324,1712.5,"{'max_depth': 4, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.25, 'subsample': 0.3, 'learning_rate': 0.01}"
98,0.107609,0.013368,1730.5,"{'max_depth': 4, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.25, 'subsample': 0.4, 'learning_rate': 0.01}"
19,0.107610,0.012640,1682.4,"{'max_depth': 4, 'colsample_bytree': 0.25, 'colsample_bylevel': 0.3, 'subsample': 0.25, 'learning_rate': 0.01}"
122,0.107800,0.012941,1904.9,"{'max_depth': 4, 'colsample_bytree': 0.3, 'colsample_bylevel': 0.2, 'subsample': 0.45, 'learning_rate': 0.01}"


In [20]:
##### # round 3: tune alpha, lambda
##################################################
#lauraepp says don't touch these unless you know what you're doing ¯\_(ツ)_/¯
#
# reg_alphas = np.logspace(-3, -2, 3)
# reg_lambdas = np.logspace(-2, 1, 4)
# grid_search_dicts = [dict(zip(['reg_alpha', 'reg_lambda'], [a, b])) 
#                      for a,b in product(reg_alphas, reg_lambdas)]
# # merge into full param dicts
# full_search_dicts = [{**current_params, **d} for d in grid_search_dicts]

# # cv and get best params
# current_params, results_df = cv_over_param_dict(df, full_search_dicts, predictors, response, kfolds, verbose=False)

In [21]:
# round 4: learning rate
learning_rates = np.logspace(-3, -1, 5)
grid_search_dicts = [{'learning_rate': lr} for lr in learning_rates]
# merge into full param dicts
full_search_dicts = [{**current_params, **d} for d in grid_search_dicts]

# cv and get best params
current_params, results_df = cv_over_param_dict(df, full_search_dicts, predictors, response, kfolds, verbose=False)


Start Time           2020-10-18 12:42:27.852208
12:44:16   0 result mean: 0.106451 std: 0.013443, iter: 16709.50
12:44:58   1 result mean: 0.105772 std: 0.013051, iter: 6322.10
12:45:13   2 result mean: 0.104301 std: 0.012940, iter: 2275.10
12:45:19   3 result mean: 0.106255 std: 0.011736, iter: 777.70
12:45:22   4 result mean: 0.111745 std: 0.009933, iter: 355.00
Start Time           2020-10-18 12:42:27.852208
End Time             2020-10-18 12:45:22.740593
0:02:54


,rmse,std,best_iter,param_dict
2,0.104301,0.012940,2275.1,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.01}"
1,0.105772,0.013051,6322.1,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.0031622776601683794}"
3,0.106255,0.011736,777.7,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.03162277660168379}"
0,0.106451,0.013443,16709.5,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.001}"
4,0.111745,0.009933,355.0,"{'max_depth': 4, 'colsample_bytree': 0.2, 'colsample_bylevel': 0.15, 'subsample': 0.3, 'learning_rate': 0.1}"


In [22]:
%%time

xgb = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=3438,
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    booster='gbtree',   
    **current_params
)    

print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds,
                          n_jobs=-1)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster='gbtree', colsample_bylevel=0.15,
             colsample_bynode=None, colsample_bytree=0.2, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=None, max_depth=4,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=3438, n_jobs=-1, num_parallel_tree=None,
             random_state=42, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=0.3, tree_method=None,
             validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.103691 (STD 0.0133)
Raw CV RMSE 18193 (STD 2461)
CPU times: user 44 ms, sys: 120 ms, total: 164 ms
Wall time: 4.48 s


## HyperOpt

HyperOpt is a Bayesian optimization algorithm by [James Bergstra et al.](https://conference.scipy.org/proceedings/scipy2013/pdfs/bergstra_hyperopt.pdf)
  - [Home page](http://hyperopt.github.io/hyperopt/)
  - [GitHub](https://github.com/hyperopt/hyperopt)
  - [HyperOpt: Bayesian Hyperparameter Optimization](https://blog.dominodatalab.com/hyperopt-bayesian-hyperparameter-optimization/), Subir Mansukhani (2019)
    

In [23]:
xgb_tune_kwargs = {
    "n_estimators": tune.loguniform(100, 10000),
    "max_depth": tune.randint(0, 5),
    'max_leaves': tune.loguniform(1, 1000),    
    "subsample": tune.quniform(0.25, 0.75, 0.01),
    "colsample_bytree": tune.quniform(0.05, 0.5, 0.01),
    "colsample_bylevel": tune.quniform(0.05, 0.5, 0.01),    
    "learning_rate": tune.loguniform(0.001, 0.1),
#     "wandb": {
#         "project": "iowa2",
#        "api_key_file": "secrets/wandb.txt",
#    }    
}

xgb_tune_params = [k for k in xgb_tune_kwargs.keys() if k != 'wandb']
xgb_tune_params

['n_estimators',
 'max_depth',
 'max_leaves',
 'subsample',
 'colsample_bytree',
 'colsample_bylevel',
 'learning_rate']

In [24]:
# refactor to give ray.tune a single function of hyperparameters to optimize

# @wandb_mixin
def my_xgb(config):
    
    # fix these configs to match calling convention
    config['max_depth'] = int(config['max_depth']) + 2   # hyperopt needs left to start at 0 but we want to start at 2
    config['max_leaves'] = int(config['max_leaves'])
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    
    xgb = XGBRegressor(
        objective='reg:squarederror',
        n_jobs=1,
        random_state=RANDOMSTATE,
        booster='gbtree',   
        scale_pos_weight=1, 
        **config,
    )
    scores = -cross_val_score(xgb, df[predictors], df[response],
                                      scoring="neg_root_mean_squared_error",
                                      cv=kfolds)
    rmse = np.mean(scores)
    tune.report(rmse=rmse)
    # wandb.log({"rmse": rmse})
    
    return {"rmse": rmse}


In [25]:
NUM_SAMPLES=128

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# to limit number of cores, uncomment and set max_concurrent 
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

analysis = tune.run(my_xgb,
                    num_samples=NUM_SAMPLES,
                    config=xgb_tune_kwargs,                    
                    name="hyperopt_xgb",
                    metric="rmse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
#                    loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                   )

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bylevel,colsample_bytree,learning_rate,max_depth,max_leaves,n_estimators,subsample,iter,total time (s),rmse
my_xgb_545a52e0,TERMINATED,,0.28,0.36,0.00316816,1,1.61122,4626.62,0.48,1,65.1132,0.108632
my_xgb_545a52e1,TERMINATED,,0.49,0.07,0.0762624,1,628.932,2853.98,0.46,1,32.9688,0.111539
my_xgb_545a52e2,TERMINATED,,0.21,0.28,0.0133193,2,8.13262,5842.93,0.67,1,75.6614,0.10645
my_xgb_545a52e3,TERMINATED,,0.37,0.09,0.00186119,4,2.06337,6877.73,0.47,1,94.9616,0.108717
my_xgb_545a52e4,TERMINATED,,0.27,0.09,0.0012315,3,62.2503,220.67,0.66,1,1.96524,8.79762
my_xgb_545a52e5,TERMINATED,,0.06,0.33,0.0756744,3,38.0067,751.724,0.65,1,8.31455,0.109163
my_xgb_545a52e6,TERMINATED,,0.18,0.49,0.0130914,1,3.09615,7563.79,0.62,1,99.2589,0.10556
my_xgb_545a52e7,TERMINATED,,0.11,0.22,0.0221534,2,11.9418,3554.09,0.56,1,36.6305,0.105246
my_xgb_545a52e8,TERMINATED,,0.31,0.07,0.00444754,4,30.8603,6918.6,0.52,1,82.3992,0.106317
my_xgb_545a52e9,TERMINATED,,0.19,0.48,0.0181471,4,412.056,2519.09,0.73,1,58.8725,0.108843


Start Time           2020-10-18 12:45:27.268857
End Time             2020-10-18 12:52:58.825080
0:07:31


In [26]:
param_cols = ['config.' + k for k in xgb_tune_params]
analysis_results_df = analysis.results_df[['rmse', 'date', 'time_this_iter_s'] + param_cols].sort_values('rmse')
analysis_results_df


,rmse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.max_leaves,config.subsample,config.colsample_bytree,config.colsample_bylevel,config.learning_rate
trial_id,,,,,,,,,,
54fc0db7,0.104472,2020-10-18_12-49-34,55.677472,4921,3,6,0.49,0.17,0.14,0.011041
562ddeea,0.104476,2020-10-18_12-52-43,83.999157,9194,4,2,0.58,0.40,0.14,0.003981
562ddee9,0.104751,2020-10-18_12-52-14,55.853276,6353,4,4,0.46,0.09,0.12,0.009103
562ddef3,0.104770,2020-10-18_12-52-58,75.016271,9988,3,15,0.42,0.15,0.28,0.008659
54fc0dca,0.104860,2020-10-18_12-50-52,76.884430,6262,6,4,0.45,0.11,0.21,0.006063
...,...,...,...,...,...,...,...,...,...,...
562ddee4,8.524087,2020-10-18_12-51-14,1.661755,161,2,553,0.27,0.29,0.29,0.001882
54fc0dc8,8.673257,2020-10-18_12-49-36,1.516865,120,2,544,0.26,0.29,0.26,0.002380
545a52e4,8.797623,2020-10-18_12-45-35,1.965245,220,5,62,0.66,0.09,0.27,0.001231


In [27]:
best_config = {z: analysis_results_df.iloc[0]['config.' + z] for z in xgb_tune_params}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
#18338 0:07:25

XGBRegressor(base_score=None, booster=None, colsample_bylevel=0.14,
             colsample_bynode=None, colsample_bytree=0.17, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.011041340734298626, max_delta_step=None,
             max_depth=3, max_leaves=6, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=4921, n_jobs=-1,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=0.49,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.104472 (STD 0.0137)
Raw CV RMSE 18338 (STD 2518)


In [28]:
# 4096 iterations
config_4096 = {'n_estimators': 2534,
 'max_depth': 3,
 'max_leaves': 22,
 'subsample': 0.42,
 'colsample_bytree': 0.14,
 'colsample_bylevel': 0.21,
 'learning_rate': 0.025422975168222197}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **config_4096
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster=None, colsample_bylevel=0.21,
             colsample_bynode=None, colsample_bytree=0.14, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.025422975168222197, max_delta_step=None,
             max_depth=3, max_leaves=22, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=2534, n_jobs=-1,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=0.42,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.103816 (STD 0.0130)
Raw CV RMSE 18215 (STD 2393)


## Optuna

Optuna is a Bayesian optimization algorithm by [Takuya Akiba, et al.](https://arxiv.org/abs/1907.10902)

 - [Home](https://optuna.org/)
 - [Using Optuna to Optimize XGBoost Hyperparameters](https://medium.com/optuna/using-optuna-to-optimize-xgboost-hyperparameters-63bfcdfd3407), Crissman Loomis, 2020



In [29]:
# optuna
NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

# identical tune args
analysis = tune.run(my_xgb,
                    num_samples=NUM_SAMPLES,
                    config=xgb_tune_kwargs,                    
                    name="optuna_xgb",
                    metric="rmse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
#                     loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                   )

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,colsample_bylevel,colsample_bytree,learning_rate,max_depth,max_leaves,n_estimators,subsample,iter,total time (s),rmse
my_xgb_7826d350,TERMINATED,,0.38,0.13,0.00186991,2,484.138,4579,0.28,1,48.9225,0.114112
my_xgb_7826d351,TERMINATED,,0.08,0.21,0.00273314,2,156.846,512.975,0.48,1,3.17766,2.86915
my_xgb_7826d352,TERMINATED,,0.13,0.48,0.00251043,5,9.7882,8328.19,0.45,1,157.106,0.10593
my_xgb_7826d353,TERMINATED,,0.18,0.26,0.00378233,0,211.711,239.916,0.34,1,2.29738,4.68391
my_xgb_7826d354,TERMINATED,,0.16,0.14,0.00138957,2,32.5473,3383.3,0.73,1,33.334,0.188716
my_xgb_7826d355,TERMINATED,,0.34,0.23,0.0349718,1,4.41961,7129.39,0.42,1,90.9695,0.108293
my_xgb_7826d356,TERMINATED,,0.15,0.27,0.00658067,5,16.3018,3548.34,0.36,1,55.0564,0.106071
my_xgb_7826d357,TERMINATED,,0.29,0.31,0.0381394,4,11.4457,561.877,0.43,1,11.8534,0.107777
my_xgb_7826d358,TERMINATED,,0.36,0.49,0.0503126,3,1.36795,5281.9,0.27,1,108.215,0.110865
my_xgb_7826d359,TERMINATED,,0.1,0.14,0.0111213,2,768.639,127.929,0.66,1,1.61095,2.81348


Start Time           2020-10-18 12:53:38.493662
End Time             2020-10-18 13:05:30.571294
0:11:52


In [30]:
param_cols = ['config.' + k for k in xgb_tune_params]
analysis_results_df = analysis.results_df[['rmse', 'date', 'time_this_iter_s'] + param_cols].sort_values('rmse')
analysis_results_df

,rmse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.max_leaves,config.subsample,config.colsample_bytree,config.colsample_bylevel,config.learning_rate
trial_id,,,,,,,,,,
790993e8,0.104773,2020-10-18_13-03-12,67.166406,6969,2,4,0.55,0.36,0.08,0.010940
7826d3af,0.105050,2020-10-18_12-58-26,53.665777,4801,7,40,0.45,0.10,0.17,0.015833
7826d385,0.105197,2020-10-18_12-56-50,80.459449,5932,5,11,0.56,0.13,0.27,0.005102
7826d3cf,0.105397,2020-10-18_12-59-31,59.980232,5160,3,11,0.35,0.40,0.16,0.006011
7826d38b,0.105545,2020-10-18_12-56-44,63.181249,4992,5,986,0.57,0.34,0.09,0.011366
...,...,...,...,...,...,...,...,...,...,...
7826d3b9,9.400380,2020-10-18_12-57-45,1.271869,115,3,5,0.65,0.31,0.35,0.001778
7826d3a0,9.446324,2020-10-18_12-57-04,1.422891,116,3,5,0.65,0.30,0.49,0.001720
7826d3d6,9.861208,2020-10-18_12-58-44,1.622687,149,5,61,0.47,0.10,0.26,0.001051


In [31]:
best_config = {z: analysis_results_df.iloc[0]['config.' + z] for z in xgb_tune_params}

xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=RANDOMSTATE,    
    verbosity=1,
    n_jobs=-1,
    **best_config
)
print(xgb)

scores = -cross_val_score(xgb, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))


XGBRegressor(base_score=None, booster=None, colsample_bylevel=0.08,
             colsample_bynode=None, colsample_bytree=0.36, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.010940179600735424, max_delta_step=None,
             max_depth=2, max_leaves=4, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=6969, n_jobs=-1,
             num_parallel_tree=None, random_state=42, reg_alpha=None,
             reg_lambda=None, scale_pos_weight=None, subsample=0.55,
             tree_method=None, validate_parameters=None, verbosity=1)

Log1p CV RMSE 0.104773 (STD 0.0132)
Raw CV RMSE 18393 (STD 2431)


# LightGBM with HyperOpt

In [32]:
lgbm_tune_kwargs = {
    "n_estimators": tune.loguniform(100, 10000),
    "max_depth": tune.randint(0, 5),
    'num_leaves': tune.loguniform(2, 1000),               # max_leaves
    "bagging_fraction": tune.quniform(0.5, 0.8, 0.01),    # subsample
    "feature_fraction": tune.quniform(0.05, 0.5, 0.01),   # colsample_bytree
    "learning_rate": tune.loguniform(0.001, 0.1),
#     "wandb": {
#         "project": "iowa",
#     }        
}

#print("wandb name:", lgbm_tune_kwargs['wandb']['name'])
lgbm_tune_params = [k for k in lgbm_tune_kwargs.keys() if k != 'wandb']
print(lgbm_tune_params)

['n_estimators', 'max_depth', 'num_leaves', 'bagging_fraction', 'feature_fraction', 'learning_rate']


In [33]:
# @wandb_mixin
def my_lgbm(config):
    
    # fix these configs 
    config['n_estimators'] = int(config['n_estimators'])   # pass float eg loguniform distribution, use int
    config['num_leaves'] = int(config['num_leaves'])
    
    lgbm = LGBMRegressor(objective='regression',
                         max_bin=200,
                         feature_fraction_seed=7,
                         min_data_in_leaf=2,
                         verbose=-1,
                         n_jobs=1,
                         # these are specified to suppress warnings
                         colsample_bytree=None,
                         min_child_samples=None,
                         subsample=None,
                         **config,
                         # early stopping params, maybe in fit
                         #early_stopping_rounds=early_stopping_rounds,
                         #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                         #num_boost_round=num_boost_round,
                         )
    
    scores = -cross_val_score(lgbm, df[predictors], df[response],
                              scoring="neg_root_mean_squared_error",
                              cv=kfolds)
    rmse=np.mean(scores)  
    tune.report(rmse=rmse)
    # wandb.log({"rmse": rmse})
    
    return {'rmse': np.mean(scores)}

In [34]:
# tune LightGBM
print("LightGBM")

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = HyperOptSearch(random_state_seed=RANDOMSTATE)
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

# lgbm_tune_kwargs['wandb']['name'] = 'hyperopt_' + xgb_tune_kwargs['wandb']['name']

analysis = tune.run(my_lgbm,
                    num_samples=NUM_SAMPLES,
                    config = lgbm_tune_kwargs,
                    name="hyperopt_lgbm",
                    metric="rmse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
#                     loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                   )

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))

Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,max_depth,n_estimators,num_leaves,iter,total time (s),rmse
my_lgbm_34ec488e,TERMINATED,,0.71,0.16,0.00149561,4,4626.62,33.7485,1,37.9934,0.110669
my_lgbm_34ec488f,TERMINATED,,0.51,0.47,0.00455539,3,2853.98,25.5793,1,19.8378,0.113596
my_lgbm_34ec4890,TERMINATED,,0.52,0.44,0.0798818,1,5842.93,364.012,1,16.8432,0.113376
my_lgbm_34ec4891,TERMINATED,,0.52,0.11,0.00110121,2,6877.73,31.7409,1,19.4243,0.117962
my_lgbm_34ec4892,TERMINATED,,0.53,0.29,0.0403208,3,220.67,311.647,1,1.39141,0.113925
my_lgbm_34ec4893,TERMINATED,,0.72,0.46,0.0270039,1,751.724,287.74,1,2.29725,0.119654
my_lgbm_34ec4894,TERMINATED,,0.79,0.3,0.0400602,2,7563.79,191.492,1,27.5515,0.11199
my_lgbm_34ec4895,TERMINATED,,0.62,0.35,0.0027847,0,3554.09,92.5782,1,252.58,0.115672
my_lgbm_34ec4896,TERMINATED,,0.52,0.2,0.0473492,4,6918.6,58.5678,1,55.018,0.109345
my_lgbm_34ec4897,TERMINATED,,0.56,0.22,0.00678644,1,2519.09,758.292,1,5.37465,0.121027


Start Time           2020-10-18 13:06:04.686172
End Time             2020-10-18 13:22:56.859301
0:16:52


In [35]:
param_cols = ['config.' + k for k in lgbm_tune_params]
analysis_results_df = analysis.results_df[['rmse', 'date', 'time_this_iter_s'] + param_cols].sort_values('rmse')
analysis_results_df

,rmse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.num_leaves,config.bagging_fraction,config.feature_fraction,config.learning_rate
trial_id,,,,,,,,,
3816ea81,0.105215,2020-10-18_13-15-25,13.219921,3769,3,38,0.71,0.05,0.008772
35b017cf,0.105718,2020-10-18_13-10-53,19.065094,5299,3,53,0.59,0.08,0.004884
3816ea82,0.106100,2020-10-18_13-15-32,19.791423,4870,3,15,0.68,0.07,0.007243
36e50752,0.106637,2020-10-18_13-13-47,65.040398,9117,4,18,0.80,0.15,0.005656
36e50748,0.106701,2020-10-18_13-12-34,2.018001,1063,2,157,0.74,0.05,0.029755
...,...,...,...,...,...,...,...,...,...
3816ea5a,0.328525,2020-10-18_13-14-08,0.820142,296,1,269,0.58,0.23,0.001645
35b017c9,0.331127,2020-10-18_13-10-28,0.626782,236,1,2,0.79,0.48,0.001902
36e50758,0.341122,2020-10-18_13-13-20,0.828035,119,4,103,0.64,0.06,0.002454


In [36]:
best_config = {z: analysis_results_df.iloc[0]['config.' + z] for z in lgbm_tune_params}

lgbm = LGBMRegressor(objective='regression',
                     max_bin=200,
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbose=-1,
                     **best_config,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

LGBMRegressor(bagging_fraction=0.71, feature_fraction=0.05,
              feature_fraction_seed=7, learning_rate=0.008771970153586162,
              max_bin=200, max_depth=3, min_data_in_leaf=2, n_estimators=3769,
              num_leaves=38, objective='regression', verbose=-1)

Log1p CV RMSE 0.105215 (STD 0.0134)
Raw CV RMSE 18475 (STD 2468)


# LightGBM with Optuna

In [37]:
# tune LightGBM
print("LightGBM")
#!conda install -y -c conda-forge lightgbm

NUM_SAMPLES=256

start_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))

algo = OptunaSearch()
# uncomment and set max_concurrent to limit number of cores
# algo = ConcurrencyLimiter(algo, max_concurrent=10)
scheduler = AsyncHyperBandScheduler()

# lgbm_tune_kwargs['wandb']['name'] = 'optuna_' + xgb_tune_kwargs['wandb']['name']

analysis = tune.run(my_lgbm,
                    num_samples=NUM_SAMPLES,
                    config = lgbm_tune_kwargs,
                    name="optuna_lgbm",
                    metric="rmse",
                    mode="min",
                    search_alg=algo,
                    scheduler=scheduler,
                    verbose=1,
#                     loggers=DEFAULT_LOGGERS + (WandbLogger, ),
                   )

end_time = datetime.now()
print("%-20s %s" % ("Start Time", start_time))
print("%-20s %s" % ("End Time", end_time))
print(str(timedelta(seconds=(end_time-start_time).seconds)))


Trial name,status,loc,bagging_fraction,feature_fraction,learning_rate,max_depth,n_estimators,num_leaves,iter,total time (s),rmse
my_lgbm_947f0afa,TERMINATED,,0.73,0.15,0.00745826,4,378.723,116.623,1,2.64396,0.128436
my_lgbm_947f0afb,TERMINATED,,0.63,0.16,0.00882491,2,5372.49,10.8689,1,17.4725,0.107584
my_lgbm_947f0afc,TERMINATED,,0.78,0.48,0.00413636,1,4312.21,867.397,1,12.5611,0.120861
my_lgbm_947f0afd,TERMINATED,,0.64,0.31,0.0421642,3,875.521,817.678,1,4.33567,0.110224
my_lgbm_947f0afe,TERMINATED,,0.63,0.31,0.00325841,2,1042.5,2.98045,1,2.86183,0.180248
my_lgbm_947f0aff,TERMINATED,,0.59,0.39,0.0495025,3,1334.72,20.6986,1,9.77302,0.111463
my_lgbm_947f0b00,TERMINATED,,0.67,0.08,0.0041036,5,666.423,786.731,1,7.19048,0.136165
my_lgbm_947f0b01,TERMINATED,,0.53,0.23,0.00319453,0,309.107,51.6049,1,10.2761,0.197955
my_lgbm_947f0b02,TERMINATED,,0.53,0.36,0.0032399,3,255.575,144.529,1,2.05967,0.234219
my_lgbm_947f0b03,TERMINATED,,0.55,0.31,0.00359279,0,6356.06,3.34617,1,22.7112,0.11497


Start Time           2020-10-18 13:23:04.029323
End Time             2020-10-18 13:30:55.069493
0:07:51


In [38]:
param_cols = ['config.' + k for k in lgbm_tune_params]
analysis_results_df = analysis.results_df[['rmse', 'date', 'time_this_iter_s'] + param_cols].sort_values('rmse')
analysis_results_df

,rmse,date,time_this_iter_s,config.n_estimators,config.max_depth,config.num_leaves,config.bagging_fraction,config.feature_fraction,config.learning_rate
trial_id,,,,,,,,,
947f0b19,0.106321,2020-10-18_13-23-35,13.021776,3546,3,209,0.59,0.07,0.007622
947f0bbd,0.107561,2020-10-18_13-28-15,32.462942,5381,0,9,0.70,0.15,0.015505
947f0afb,0.107584,2020-10-18_13-23-25,17.472530,5372,2,10,0.63,0.16,0.008825
947f0b9b,0.108163,2020-10-18_13-27-07,18.693768,2387,4,27,0.76,0.20,0.028121
947f0b4b,0.108308,2020-10-18_13-24-46,4.617636,448,5,125,0.51,0.13,0.050450
...,...,...,...,...,...,...,...,...,...
947f0bc6,0.339327,2020-10-18_13-28-02,0.792957,169,1,2,0.51,0.11,0.002616
947f0b12,0.345373,2020-10-18_13-23-20,0.657763,201,1,32,0.55,0.10,0.001998
947f0b47,0.349510,2020-10-18_13-24-28,0.830739,143,3,11,0.69,0.30,0.001223


In [39]:
best_config = {z: analysis_results_df.iloc[0]['config.' + z] for z in lgbm_tune_params}

lgbm = LGBMRegressor(objective='regression',
                     max_bin=200,
                     feature_fraction_seed=7,
                     min_data_in_leaf=2,
                     verbosfe=-1,
                     **best_config,
                     # early stopping params, maybe in fit
                     #early_stopping_rounds=early_stopping_rounds,
                     #valid_sets=[xgtrain, xgvalid], valid_names=['train','valid'], evals_result=evals_results
                     #num_boost_round=num_boost_round,
                     )
 
print(lgbm)

scores = -cross_val_score(lgbm, df[predictors], df[response],
                          scoring="neg_root_mean_squared_error",
                          cv=kfolds)

raw_scores = [cv_to_raw(x) for x in scores]
print()
print("Log1p CV RMSE %.06f (STD %.04f)" % (np.mean(scores), np.std(scores)))
print("Raw CV RMSE %.0f (STD %.0f)" % (np.mean(raw_scores), np.std(raw_scores)))
raw_scores = [cv_to_raw(x) for x in scores]

LGBMRegressor(bagging_fraction=0.59, feature_fraction=0.07,
              feature_fraction_seed=7, learning_rate=0.007622193956315748,
              max_bin=200, max_depth=3, min_data_in_leaf=2, n_estimators=3546,
              num_leaves=209, objective='regression', verbosfe=-1)

Log1p CV RMSE 0.106321 (STD 0.0133)
Raw CV RMSE 18679 (STD 2465)


# Ray Cluster on AWS
(Google Cloud Platform and Azure should be supported as well, see [Ray docs and examples](https://github.com/ray-project/ray/tree/master/python/ray/autoscaler))

- Clusters are defined in `ray1.1.yaml`
- boto3 and AWS CLI configured credentials are used, so [install and configure AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html)
- Edit `ray1.1.yaml` file with your region, availability zone, subnet, imageid information
    - to get those variables launch the latest Deep Learning AMI (Ubuntu 18.04) Version 35.0 into a small instance in your favorite region/zone
    - test that it works
    - note those 4 variables: region, availability zone, subnet, AMI imageid
    - terminate the instance and edit `ray1.1.yaml` accordingly
    - in future you can create your own image with everything pre-installed and specify its AMI imageid, instead of using the generic image and installing everything at launch.
- To run the cluster: 
`ray up ray1.1.yaml`
    - Creates head instance using image specified.
    - Installs ray and related requirements
    - Clones this Iowa repo
    - Launches worker nodes per auto-scaling parameters (currently we fix the number of nodes because we're not benching the time the cluster will take to auto-scale)
- After cluster starts you can check AWS console and note that several instances launched.
- Check `ray monitor ray1.1.yaml` for any error messages
- Run Jupyter on the cluster with port forwarding
 `ray exec ray1.1.yaml --port-forward=8899 'source ~/anaconda3/bin/activate tensorflow_p36 && jupyter notebook --port=8899'`
- Open the notebook on the generated URL e.g. http://localhost:8899/?token=5f46d4355ae7174524ba71f30ef3f0633a20b19a204b93b4
- Make sure to hoose the default kernel to make sure it runs in the conda environment with all installs
- Make sure to use the ray.init() command given in the startup messages.
- You can also run a terminal on the head node of the cluster with
 `ray attach /Users/drucev/projects/iowa/ray1.1.yaml`
- You can also ssh explicitly with the IP address and the generated private key
 `ssh -o IdentitiesOnly=yes -i ~/.ssh/ray-autoscaler_1_us-east-1.pem ubuntu@54.161.200.54`
- run port forwarding to the Ray dashboard with   
`ray dashboard ray1.1.yaml`
and then open
 http://localhost:8265/
- the cluster will incur AWS charges so `ray down ray1.1.yaml` when complete
- Other than connecting to Ray cluster, runs identically
- see hyperparameter_optimization.ipynb, separated out so each notebook can be run end-to-end with/without cluster setup

see https://docs.ray.io/en/latest/cluster/launcher.html for additional info

## Concluding remarks

We observe a modest but non-negligeable improvement in the target metric with a less manual process vs. sequential tuning.
hyoperopt.optuna yes
ray maybe, depends on further work compare w/optuna and hyperopt online. depends on integration between ray, early stopping and search algo
cluster , in general no need and costs add up. MacBook Pro w/16 threads and desktop with GPU are plenty.

I intend to use HyperOpt and Optuna for xgboost going forward, no more grid search for me! In every case I've applied them, I've gotten at least a small improvement in the best metrics I found using grid search methods. Additionally, it's fire and forget (although with a little elbow grease the 4-pass sequential grid search could be made fire and forget.)

These two algorithms seem to be the most popular but I may try the other algos systematically. 

I am surprised that Elasticnet, i.e. regularized linear regression outperforms boosting. 
This dataset has been heavily engineered so that linear methods work well. Predictors were chosen using lasso/elasticnet and we used log and Box-Cox transforms to force predictors to follow assumptions of least-squares.  

This tends to validate one of the [critiques of machine learning](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3624052), that the most powerful ML methods don't necessarily converge all the way to the best solution. If you have a ground truth that is linear plus noise, a complex XGBoost or neural network algorithm should get arbitrarily close to the closed-form optimal solution but will never match it exactly. XGBoost is piecewise constant and the complex neural network is subject to the vagaries of stochastic gradient descent. 

But Elasticnet with L1 + L2 regularization plus gradient descent and hyperparameter optimization is still machine learning. It's simply the form best matched to the problem. In the real world where things don't match assumptions of least-squares, boosting performs extremely well. And even on this dataset, engineered for the linear models, SVR and KernelRidge performed better than Elasticnet (not shown) and ensembling Elasticnet with XGBoost, LightGBM, SVR, neural networks worked best of all. 

To paraphrase Casey Stengel, clever feature engineering will always outperform clever model algorithms and vice-versa<sup>*</sup>. But improving your hyperparameters with these best practices will always improve your results.

<sup>*</sup>This is not intended to make sense.
